***GENERATED CODE FOR stagestestlr PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def renameColumns(df, functionsData, applyOn):
    for functionData in functionsData:
        df = df.rename(
            columns={functionData['oldName']: functionData['newName']})
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyspark


def linear_regression(sparkDF, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lr_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    lr_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lr_model.score(X_test, y_test))

    data = {'model': lr_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
#%run stagestestlrHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': '172.17.0.6', 'port': '1433', 'password': 'num_sql@1433', 'database': 'testdb', 'user': 'SA', 'qtype': 'dbtable', 'query': 'BostonTrain', 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run stagestestlrHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(sqlserver,spark,json.dumps( {"url": null, "source_attributes": {"host": "172.17.0.6", "port": "1433", "password": "num_sql@1433", "database": "testdb", "user": "SA", "qtype": "dbtable", "query": "BostonTrain", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}, "DataPrepFile": null, "data_source": "mssql", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/606dacf1e79af4b77a0ea7fa1617800685/0part.csv", "requestRatio": 1.0, "totalRows": 506, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 15, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 88.9762, "min": 0.00632, "mean": 3.6135235573122535, "missing": 0.0, "stddev": 8.6}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 100.0, "min": 0.0, "mean": 11.363636363636363, "missing": 0.0, "stddev": 23.32}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 27.74, "min": 0.46, "mean": 11.136778656126504, "missing": 0.0, "stddev": 6.86}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.0691699604743083, "missing": 0.0, "stddev": 0.25}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.871, "min": 0.385, "mean": 0.5546950592885372, "missing": 0.0, "stddev": 0.12}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 8.78, "min": 3.5610000000000004, "mean": 6.284634387351788, "missing": 0.0, "stddev": 0.7}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 100.0, "min": 2.9, "mean": 68.57490118577078, "missing": 0.0, "stddev": 28.15}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 12.1265, "min": 1.1296, "mean": 3.795042687747034, "missing": 0.0, "stddev": 2.11}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "real", "max": 24.0, "min": 1.0, "mean": 9.549407114624506, "missing": 0.0, "stddev": 8.71}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "real", "max": 711.0, "min": 187.0, "mean": 408.2371541501976, "missing": 0.0, "stddev": 168.54}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 22.0, "min": 12.6, "mean": 18.455533596837967, "missing": 0.0, "stddev": 2.16}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 0.32, "mean": 356.67403162055257, "missing": 0.0, "stddev": 91.29}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 37.97, "min": 1.73, "mean": 12.653063241106723, "missing": 0.0, "stddev": 7.14}, {"column": "MEDV", "alias": "MEDV", "generated": 0, "type": "real", "max": 50.0, "min": 5.0, "mean": 22.532806324110698, "missing": 0.0, "stddev": 9.2}, {"column": "zn", "alias": "zn", "generated": 0, "type": "real", "max": 100.0, "min": 0.0, "mean": 11.363636363636363, "missing": 0.0, "stddev": 23.32}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "editname", "applyOn": [{"columnName": "zn", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}], "functionsData": [{"oldName": "ZN", "newName": "zn", "column": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}]}], "functionChanges": [{"columnName": "zn", "functionName": "Edit Column Name", "Type": "real", "Parameters": [{"oldName": "ZN", "newName": "zn", "column": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "generated": 0, "position": 14, "type": "real"}, {"field": "zn", "alias": "zn", "generated": 0, "position": 15, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run stagestestlrHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "2.33", "stddev": "5.05", "min": "0.01432", "max": "22.5971", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.3", "stddev": "21.95", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "9.55", "stddev": "5.79", "min": "1.32", "max": "19.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.04", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.54", "stddev": "0.1", "min": "0.41100000000000003", "max": "0.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "6.48", "stddev": "0.77", "min": "5.0", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "72.41", "stddev": "24.8", "min": "17.2", "max": "98.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "3.76", "stddev": "2.09", "min": "1.5184", "max": "8.3248", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.96", "stddev": "7.85", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "399.52", "stddev": "149.09", "min": "193.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "18.38", "stddev": "1.99", "min": "14.7", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "340.71", "stddev": "114.3", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "11.52", "stddev": "6.86", "min": "2.98", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "22.64", "stddev": "10.65", "min": "7.4", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "zn14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.3", "stddev": "21.95", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "zn14"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run stagestestlrHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=linear_regression(autofe, ["CRIM", "ZN1", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "zn14"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
